In [9]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

In [10]:
housing=fetch_california_housing()

m, n=housing.data.shape
batchsize=100
n_batches=np.ceil(m/batchsize).astype(int)
n_batches

stdsc=StandardScaler()
scaled_housing=stdsc.fit_transform(housing.data)
scaled_housing_plus_bias=np.c_[np.ones([m,1]),scaled_housing]

In [11]:
scaled_housing_plus_bias.mean(axis=0)

array([ 1.00000000e+00,  6.60969987e-17,  5.50808322e-18,  6.60969987e-17,
       -1.06030602e-16, -1.10161664e-17,  3.44255201e-18, -1.07958431e-15,
       -8.52651283e-15])

In [12]:
def select_minibatch(epoch, batch_index, batch_size):
    np.random.seed(epoch*n_batches+batch_index)
    indices=np.random.randint(m, size=batch_size)
    X=scaled_housing_plus_bias[indices]
    y=housing.target[indices]
    #print(indices)
    return X,y

X,y=select_minibatch(1, batchsize, 5)
X.shape

(5, 9)

In [13]:
select_minibatch(1,2,3)

(array([[ 1.        ,  0.08455156,  0.50539419, -0.57887605,  0.0191899 ,
         -0.26445388, -0.1219134 , -1.33988509,  1.15783575],
        [ 1.        ,  1.25053131, -1.95780625,  0.41746511, -0.12133355,
          0.15323025,  0.02178475, -0.92788091,  0.98314   ],
        [ 1.        ,  0.69893988,  0.10810379,  0.36583982, -0.16371001,
          0.60800262, -0.05276264, -0.76401561,  0.76352248]]),
 array([3.853, 2.339, 2.792]))

In [14]:
X=tf.placeholder(shape=(None,n+1), dtype=tf.float32)
y=tf.placeholder(shape=(None,1), dtype=tf.float32)

#tf.set_random_seed(42)
theta=tf.Variable(tf.random_uniform((n + 1, 1), -1.0, 1.0, seed=42),name='Theta')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

error=y-tf.matmul(X,theta)
mse=tf.reduce_mean(tf.square(error), name='mse')

lerr=0.01
optimizer=tf.train.GradientDescentOptimizer(learning_rate=lerr)
training_op=optimizer.minimize(mse)


In [15]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(theta.eval())
    n_epochs=10
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            Xb,yb=select_minibatch(epoch, batchsize, batch_index)
            sess.run(training_op, feed_dict={X:Xb, y:yb.reshape(-1,1)})
    best_theta=theta.eval(session=sess)

[[ 0.9045429 ]
 [ 0.35481548]
 [ 0.5906365 ]
 [ 0.51156354]
 [-0.04808879]
 [ 0.26202965]
 [-0.62795925]
 [-0.7713845 ]
 [-0.32755637]]


In [16]:
best_theta

array([[ 2.0715818 ],
       [ 0.82431453],
       [ 0.15347598],
       [-0.13969152],
       [ 0.26221293],
       [-0.04512911],
       [-1.1565139 ],
       [-0.9987298 ],
       [-0.88783884]], dtype=float32)